In [1]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

GPU available: True
Device name: Tesla T4


In [1]:
#Install Libraries
!pip install -q twilio pydub
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install faster-whisper

  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-ajkgxzwl
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-ajkgxzwl
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-15pex0i8/descript-audiotools_2386a575e08b43c09f6b62fb546005c9
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-15pex0i8/descript-audiotools_2386a575e08b43c09f6b62fb546005c9
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
  Using cached faster_whisper-1.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached o

In [2]:
# Import Dependencies
import json
import time
import requests
from twilio.rest import Client
from pydub import AudioSegment
from faster_whisper import WhisperModel

In [3]:
# Twilio Setup -  for calling and recording you can get all these info when you create an account on twilio
TWILIO_ACCOUNT_SID = 'ACcec734d6772e3250a14ed21df94d09ce'     # Your Twilio SID
TWILIO_AUTH_TOKEN = 'cc0e77e4c6da4978f4914c27ab70ae62'      # Your Twilio Auth Token
TWILIO_PHONE_NUMBER = '+16073605150'    # Your Twilio Number (e.g., +1415xxxxxx)
TWIML_BIN_URL = 'https://handler.twilio.com/twiml/EH1d7ad9b4bb5da53e3270b1579d7878fd' #url created in twiml bin in xml for twilio calling tone and audio like what to speak

client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)

In [5]:
# this will ask for candidate's mobile number then Triggers the Call
CANDIDATE_PHONE = input("Enter candidate number (e.g., +91xxxxxxxxxx): ")
call = client.calls.create(
    to=CANDIDATE_PHONE,
    from_=TWILIO_PHONE_NUMBER,
    url=TWIML_BIN_URL
)

print("Call initiated. SID:", call.sid)

Enter candidate number (e.g., +91xxxxxxxxxx): +919870901120
Call initiated. SID: CAa94c74a889f4d651ac4bbfe5b5b18f0b


In [6]:
print("Waiting 20 seconds for call and recordings to complete...")
time.sleep(20)

# Fetch recordings
recordings = client.recordings.list(call_sid=call.sid)
print(f"\n Found {len(recordings)} recording(s)")

# Load Whisper model
model = WhisperModel("medium", compute_type="float16")

conversation_log = []

# Process & Transcribe each recording
for idx, rec in enumerate(recordings):
    audio_url = f"https://api.twilio.com{rec.uri.replace('.json', '.wav')}"
    response = requests.get(audio_url, auth=(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN))

    # Save original recording
    raw_file = f"response_{idx+1}.wav"
    with open(raw_file, "wb") as f:
        f.write(response.content)

    # Resample to 16kHz mono
    sound = AudioSegment.from_file(raw_file)
    sound = sound.set_frame_rate(16000).set_channels(1)
    processed_file = f"processed_{idx+1}.wav"
    sound.export(processed_file, format="wav")

    # Transcribe using Whisper
    segments, _ = model.transcribe(processed_file, beam_size=5)
    candidate_response = " ".join([seg.text.strip() for seg in segments])

    conversation_log.append({
        "agent": f"Q{idx+1}",
        "candidate": candidate_response
    })

# Print Final Conversation Log
print("\n Final Transcription Log:\n")
for turn in conversation_log:
    print(f"{turn['agent']}: {turn['candidate']}")

Waiting 20 seconds for call and recordings to complete...

 Found 5 recording(s)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]


 Final Transcription Log:

Q1: I am located in Gurgaon and I can plan within 7 days.
Q2: I have around 10 months of experience as of now.
Q3: My primary technical skills are Python, SQL, W, Power BI, Excel, EWS, basics, and yeah.
Q4: Myself Navdeep Shishodiya. I have currently completed my graduation in B.Tech data science in 2025 and now I'm doing some internships in a company called Spine. Currently my role is AI intern.
Q5: My name is Navdeep Shishodiya.


In [7]:
# LLM Prompt for candidate's review
prompt = f"""
You are a recruitment AI assistant. Here's the transcript of a candidate interview:

---
{conversation_log}
---

Your tasks:
1. Detect sentiment and tone (positive, negative, neutral, confident, hesitant, confused).
2. Identify key information: skills, experience, location, willingness to join.
3. Determine if the candidate asked a question that needs human escalation and note the questions.
4. Recommend a decision: Shortlist, Escalate, or Hold.
5. Suggest a closing response line.

Respond in JSON format with keys:
{{
  "Sentiment": "",
  "Tone": "",
  "Skills": [],
  "Experience": "",
  "Location": "",
  "Willing_to_Join": true,
  "Candidate_Asked_Question": false,
  "Candidate_Asked_Questions_List": [],
  "Decision": "",
  "Closing_Line": ""
}}
"""

In [8]:
# Call LLM
openrouter_api_key = "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" #Enter OpenRouter API Key
model = "mistralai/devstral-small" # can use different models

headers = {
    "Authorization": f"Bearer {openrouter_api_key}",
    "Content-Type": "application/json"
}

payload = {
    "model": model,
    "messages": [
        {"role": "system", "content": "You are a helpful recruitment AI assistant."},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=payload)

In [9]:
import re

try:
    llm_output_raw = response.json()["choices"][0]["message"]["content"]

    # Strip markdown-style formatting like ```json
    llm_output_clean = re.sub(r"```json|```", "", llm_output_raw).strip()

    # Convert to dictionary
    final_output = json.loads(llm_output_clean)

    # Pretty Output (like a report)
    print("\n==============================")
    print(" Candidate Screening Summary")
    print("==============================\n")

    print(f"-> Sentiment       : {final_output.get('Sentiment', 'N/A')}")
    print(f"-> Tone            : {final_output.get('Tone', 'N/A')}")
    print(f"-> Experience      : {final_output.get('Experience', 'N/A')}")
    print(f"-> Skills          : {', '.join(final_output.get('Skills', []))}")
    print(f"-> Location        : {final_output.get('Location', 'N/A')}")
    print(f"-> Willing to Join : {'Yes' if final_output.get('Willing_to_Join') else 'No'}")
    print(f"-> Asked Questions : {'Yes' if final_output.get('Candidate_Asked_Question') else 'No'}")

    if final_output.get("Candidate_Asked_Questions_List"):
        print(" Questions from Candidate:")
        for q in final_output["Candidate_Asked_Questions_List"]:
            print(f"   - {q}")

    print(f"\n Final Decision  : {final_output.get('Decision', 'N/A')}")
    print(f" Closing Line    : {final_output.get('Closing_Line', 'N/A')}")

except Exception as e:
    print("\n LLM Parsing Error:", e)

    if 'response' in locals():
        print("\n Raw response:")
        try:
            print(response.text)
        except:
            print("No .text content in response.")
    else:
        print("No response object was captured.")



 Candidate Screening Summary

-> Sentiment       : Neutral
-> Tone            : Confident
-> Experience      : 10 months
-> Skills          : Python, SQL, W, Power BI, Excel, EWS, basics
-> Location        : Gurgaon
-> Willing to Join : Yes
-> Asked Questions : No

 Final Decision  : Shortlist
 Closing Line    : Thank you for your time, Navdeep. We will be in touch soon regarding the next steps in the recruitment process.
